In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

data = pd.read_csv("../datasets/vgsales_clean.csv")

data.head()

,Nombre,Plataforma,Año,Género,Publicador,Ventas_Norteamérica,Ventas_Europa,Ventas_Japón,Ventas_Otros,Ventas_Globales
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### DESCRIPCION DE LA HIPOTESIS PRINCIPAL
Queremos comprobar si las ventas globales hacen alguna distincion dependiendo del genero, y para esto es necesario comprobar la normalidad entre cada grupo de genero con sus ventas globales, para despues hacer un uso de algun metodo ya sea para pruebas normalizadas o parametricas. 

### PRUEBA DE NORMALIDAD
En el siguiente casilla usamos la prueba Shapiro-Wilk que se encuentra en la libreria stats. Hicimos uso de un agrupamiento para extraer los datos de ventas globales por cada genero de videojuegos almacenados, una vez ahi verificamos si la cantidad de valores es mayor que 3, esto por un requerimiento estadistico de la prueba shapiro. Entonces pasamos los valores a travez de la prueba shapiro y nos lanza los siguientes datos: 

In [26]:
for genero, grupo in data.groupby("Género"):
    valores = grupo["Ventas_Globales"].values
    if len(valores) >= 3:  
        stat, p_value = stats.shapiro(valores)
        if p_value < 0.05:
            print(f"Género: {genero:<20} p-value: {p_value:.2e}    LOS DATOS NO SON NORMALES")
        elif p_value >= 0.05:
            print(f"Género: {genero:<20} p-value: {p_value:.2e}    LOS DATOS SON NORMALES")

Género: Action               p-value: 4.00e-61    LOS DATOS NO SON NORMALES
Género: Adventure            p-value: 2.41e-52    LOS DATOS NO SON NORMALES
Género: Fighting             p-value: 8.50e-38    LOS DATOS NO SON NORMALES
Género: Misc                 p-value: 3.49e-57    LOS DATOS NO SON NORMALES
Género: Platform             p-value: 6.72e-41    LOS DATOS NO SON NORMALES
Género: Puzzle               p-value: 3.84e-41    LOS DATOS NO SON NORMALES
Género: Racing               p-value: 3.30e-46    LOS DATOS NO SON NORMALES
Género: Role-Playing         p-value: 9.97e-55    LOS DATOS NO SON NORMALES
Género: Shooter              p-value: 9.84e-45    LOS DATOS NO SON NORMALES
Género: Simulation           p-value: 2.91e-45    LOS DATOS NO SON NORMALES
Género: Sports               p-value: 4.32e-61    LOS DATOS NO SON NORMALES
Género: Strategy             p-value: 1.99e-38    LOS DATOS NO SON NORMALES


La prueba shapiro de normalidad consta de 2 hipotesis<br>
H0 = La distribucion es normal<br>
H1 = La distribucion no es normal<br>
Esto esta medido a travez de un estadistico de prueba aqui dado por p-value.<br>
Para que la hipotesis H0 sea considerada correcta, el p-value debe de ser mayor que 0.05, lo cual es falso en todos los generos con los que la probamos. 

Ahora que ya sabemos que no son datos normalizados tenemos 2 opciones, normalizarlos o usar alguna prueba que no requiera datos normalizados. En esta practica hemos elegido usar kruskall wallis test el cual es una prueba parametrica que no hace uso de datos normalizados. 

In [28]:
grupos = [grupo["Ventas_Globales"].values 
          for nombre, grupo in data.groupby("Género")]

# Aplicar Kruskal-Wallis
h_stat, p_value = stats.kruskal(*grupos)

print("Estadístico H:", h_stat)
if p_value < 0.05:
        print(f"p-value: {p_value:.2e}    hay diferencias significativas entre al menos un par de géneros")
elif p_value >= 0.05:
        print(f"p-value: {p_value:.2e}    no hay evidencia suficiente para decir que los géneros difieran en ventas globales")


Estadístico H: 836.6730720086949
p-value: 2.52e-172    hay diferencias significativas entre al menos un par de géneros


La prueba Kruskall wallis hace uso de 2 hipotesis:<br>
H0: hay al menos 2 grupos que difieren en cuanto a ventas<br>
H1: todos los grupos se parecen en cuanto a ventas<br>
todo se basa en el valor de p-value, como obtuvimos practicamente un p-value = 0, esto lo que nos lanza es que al menos 2 grupos difiren en cuanto ventas.<br>

#### Conclusion:<br>
#### AL menos un genero de los 12 es mas comprado

In [32]:
# Codigo en funciones
def prueba_normalizacion(df):
    """Hace uso de la prueba Shapiro para comprobar la normalidad de los datos"""
    for genero, grupo in df.groupby("Género"):
        valores = grupo["Ventas_Globales"].values
        if len(valores) >= 3:  
            #Prueba shapiro
            stat, p_value = stats.shapiro(valores)
            if p_value < 0.05:
                print(f"Género: {genero:<20} p-value: {p_value:.2e}    LOS DATOS NO SON NORMALES")
            elif p_value >= 0.05:
                print(f"Género: {genero:<20} p-value: {p_value:.2e}    LOS DATOS SON NORMALES")

def prueba_grupos(df):
    """HAce uso de la prueba Kruskal-Wallis para comprobar la diferencias entre grupos"""
    grupos = [grupo["Ventas_Globales"].values 
          for nombre, grupo in df.groupby("Género")]

    # Kruskal-Wallis
    h_stat, p_value = stats.kruskal(*grupos)
    print("Estadístico H:", h_stat)
    if p_value < 0.05:
            print(f"p-value: {p_value:.2e}    hay diferencias significativas entre al menos un par de géneros")
    elif p_value >= 0.05:
            print(f"p-value: {p_value:.2e}    no hay evidencia suficiente para decir que los géneros difieran en ventas globales")

#USO
prueba_normalizacion(data)
prueba_grupos(data)

Género: Action               p-value: 4.00e-61    LOS DATOS NO SON NORMALES
Género: Adventure            p-value: 2.41e-52    LOS DATOS NO SON NORMALES
Género: Fighting             p-value: 8.50e-38    LOS DATOS NO SON NORMALES
Género: Misc                 p-value: 3.49e-57    LOS DATOS NO SON NORMALES
Género: Platform             p-value: 6.72e-41    LOS DATOS NO SON NORMALES
Género: Puzzle               p-value: 3.84e-41    LOS DATOS NO SON NORMALES
Género: Racing               p-value: 3.30e-46    LOS DATOS NO SON NORMALES
Género: Role-Playing         p-value: 9.97e-55    LOS DATOS NO SON NORMALES
Género: Shooter              p-value: 9.84e-45    LOS DATOS NO SON NORMALES
Género: Simulation           p-value: 2.91e-45    LOS DATOS NO SON NORMALES
Género: Sports               p-value: 4.32e-61    LOS DATOS NO SON NORMALES
Género: Strategy             p-value: 1.99e-38    LOS DATOS NO SON NORMALES
Estadístico H: 836.6730720086949
p-value: 2.52e-172    hay diferencias significativas en